In [1]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'rag_from_scratch_1_to_4'

In [2]:
def set_secret_from_file(secret_file_path: str, env_var_name: str) -> None:
    """
    Reads a secret from a file and sets it as an environment variable.
    
    Args:
        secret_file_path (str): The path to the file containing the secret.
        env_var_name (str): The name of the environment variable to set.
    """
    with open(secret_file_path, 'r') as file:
        secret = file.read().strip()
        os.environ[env_var_name] = secret


set_secret_from_file('data/secret.txt', 'LANGCHAIN_API_KEY')

In [3]:
! pip install -q -U langchain_community tiktoken chromadb langchain langchainhub sentence_transformers ipywidgets bs4

In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
#loader = WebBaseLoader(
#    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#    bs_kwargs=dict(
#        parse_only=bs4.SoupStrainer(
#            class_=("post-content", "post-title", "post-header")
#        )
#    ),
#)
data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [6]:
from langchain.embeddings import SentenceTransformerEmbeddings
from tqdm.autonotebook import tqdm, trange

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/c6/sp6yjxnd6934fjnjcc_rmb100000gp/T/ipykernel_23031/776569437.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/var/folders/c6/sp6yjxnd6934fjnjcc_rmb100000gp/T/ipykernel_23031/776569437.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/donbuddenbaum/anaconda3/envs/ragfs/lib/python3.12/site-packages/torch/backends/mps/__init__.py:22: UserWarning: Skipping device NVIDIA GeForce GTX 675MX that does not support Metal 

In [7]:
# Add to ChromaDB vector store
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

In [8]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [9]:
docs[0]

Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.')

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM

model_local = ChatOllama(base_url='http://192.168.2.39:11434', model='gemma2', temperature=0, timeout=180)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

/var/folders/c6/sp6yjxnd6934fjnjcc_rmb100000gp/T/ipykernel_23031/2048675709.py:16: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model_local = ChatOllama(base_url='http://192.168.2.39:11434', model='gemma2', temperature=0, timeout=180)


In [11]:
# Question
chain.invoke("What are the types of agent memory?")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'The provided text states that there are different "Types of Memory" but doesn\'t specify what they are.  \n'

In [12]:
chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down large tasks into smaller, more manageable subgoals. This enables efficient handling of complex tasks.  \n'